In [ ]:
# ==========================================================
# ETL DE 3 CAPAS EN DATABRICKS (BRONZE → SILVER → GOLD)
# ==========================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, to_timestamp, round as spark_round

# Iniciar sesión Spark
spark = SparkSession.builder.appName("ETL_3_Capas_Databricks").getOrCreate()


In [ ]:
# ==========================================================
# CAPA BRONZE - LECTURA DE LA TABLA CREADA
# ==========================================================
df_bronze = spark.table("default.pacientesbronce")

print("Vista previa de Bronze:")
df_bronze.show(5)

In [ ]:
# ==========================================================
# CAPA SILVER → Limpieza y validación
# ==========================================================

df_silver = (
    df_bronze
        .dropna(subset=["id", "nombre", "edad", "obra_social", "fecha_turno"])
        .filter(col("edad") > 0)
        .withColumn("fecha_turno", to_timestamp("fecha_turno"))
        .dropDuplicates(["id"])
)

# Guardar capa SILVER como tabla dentro de default
df_silver.write.mode("overwrite").saveAsTable("default.pacientessilver")

print("Capa SILVER creada correctamente")


In [ ]:
# ==========================================================
# CAPA GOLD → Agregación y métricas
# ==========================================================

df_gold = (
    df_silver.groupBy("obra_social")
             .agg(
                 count("*").alias("total_pacientes"),
                 spark_round(avg("edad"), 2).alias("edad_promedio")
             )
)

# Guardar capa GOLD como tabla dentro de default
df_gold.write.mode("overwrite").saveAsTable("default.pacientesgold")

print("Capa GOLD creada correctamente")


In [ ]:
# ==========================================================
# Verificación final
# ==========================================================
spark.sql("SELECT * FROM default.pacientesgold").show()